In [1]:
import pandas as pd
import numpy as np
import json
import traceback, sys
from tqdm import tqdm

In [2]:
# Constants
# IGNORE_TIME = 15.9e9  # Time in nanoseconds to ignore
START_TIME = 0
END_TIME = 15.5e9

# Helper Functions
def filter_data_between_times(df, start_time, end_time, time_column='time'):
    """Filter the DataFrame to ignore rows before a certain time"""
    # return df[df[time_column] > IGNORE_TIME]
    return df[(df[time_column] > start_time) & (df[time_column] < end_time)]

In [3]:
BASE_PATH = "/Users/shamweelmohammed/Desktop/Masters/Research/trace/analysis/output/june_7/model_loading/"

In [4]:
REPORTS_PATH = "/Users/shamweelmohammed/Desktop/Masters/Research/trace/apr19/reports_inference_profile_llama_apr19_3_bsz_1/"

# GPU Projection

In [5]:
# csv_path = "/Users/shamweelmohammed/Desktop/Masters/Research/trace/apr19/nvtx_pushpop_trace.csv"
nvtx_gpu_proj_trace_csv_path = REPORTS_PATH + "nvtx_gpu_proj_trace.csv"

# Load the CSV data into a DataFrame
df_nvtx_gpu_proj_trace = pd.read_csv(nvtx_gpu_proj_trace_csv_path)

In [6]:
# Add index column
df_nvtx_gpu_proj_trace['index'] = np.arange(len(df_nvtx_gpu_proj_trace))
df_nvtx_gpu_proj_trace

,Name,Projected Start (ns),Projected Duration (ns),Orig Start (ns),Orig Duration (ns),Style,PID,TID,NumGPUOps,Lvl,NumChild,RangeId,ParentId,RangeStack,index
0,{'traceMarker': ['/home/llama/example_text_com...,9255113734,58108556024,2804473360,69103057055,PushPop,734,734,88119,0,1,3877,NaN,:3877,0
1,"{'mod': 'torch', 'op': '__getattr__', 'args': []}",9255113734,58108556024,2804485120,69103045295,PushPop,734,734,88119,1,1,3878,3877.0,:3877:3878,1
2,{'traceMarker': ['/home/llama/example_text_com...,9255113734,58108556024,2804678389,69102852026,PushPop,734,734,88119,2,1,3879,3878.0,:3877:3878:3879,2
3,"{'mod': 'torch', 'op': '__getattr__', 'args': []}",9255113734,58108556024,2804686929,69102843486,PushPop,734,734,88119,3,1,3880,3879.0,:3877:3878:3879:3880,3
4,{'traceMarker': ['/home/llama/example_text_com...,9255113734,58108556024,2804866478,69102663937,PushPop,734,734,88119,4,1,3881,3880.0,:3877:3878:3879:3880:3881,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368899,"aten::item, op_id = 458083",67363390493,2496,67363371758,241859,PushPop,734,734,1,1417,1,981615,981614.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,368899
368900,"aten::_local_scalar_dense, op_id = 458084",67363390493,2496,67363372998,240149,PushPop,734,734,1,1418,0,981616,981615.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,368900
368901,"aten::to, op_id = 458087",67363666974,2784,67363632037,213899,PushPop,734,734,1,1416,1,981622,6462.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,368901
368902,"aten::_to_copy, op_id = 458088",67363666974,2784,67363633597,211919,PushPop,734,734,1,1417,1,981623,981622.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,368902


In [7]:
# df_nvtx_gpu_proj_trace_filtered = filter_data_after_time(df_nvtx_gpu_proj_trace, 'Orig Start (ns)')
df_nvtx_gpu_proj_trace_filtered = filter_data_between_times(df_nvtx_gpu_proj_trace, START_TIME, END_TIME, 'Orig Start (ns)')

In [8]:
df_nvtx_gpu_proj_trace_filtered

,Name,Projected Start (ns),Projected Duration (ns),Orig Start (ns),Orig Duration (ns),Style,PID,TID,NumGPUOps,Lvl,NumChild,RangeId,ParentId,RangeStack,index
0,{'traceMarker': ['/home/llama/example_text_com...,9255113734,58108556024,2804473360,69103057055,PushPop,734,734,88119,0,1,3877,NaN,:3877,0
1,"{'mod': 'torch', 'op': '__getattr__', 'args': []}",9255113734,58108556024,2804485120,69103045295,PushPop,734,734,88119,1,1,3878,3877.0,:3877:3878,1
2,{'traceMarker': ['/home/llama/example_text_com...,9255113734,58108556024,2804678389,69102852026,PushPop,734,734,88119,2,1,3879,3878.0,:3877:3878:3879,2
3,"{'mod': 'torch', 'op': '__getattr__', 'args': []}",9255113734,58108556024,2804686929,69102843486,PushPop,734,734,88119,3,1,3880,3879.0,:3877:3878:3879:3880,3
4,{'traceMarker': ['/home/llama/example_text_com...,9255113734,58108556024,2804866478,69102663937,PushPop,734,734,88119,4,1,3881,3880.0,:3877:3878:3879:3880:3881,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,"{'mod': 'Tensor', 'op': 'copy_', 'args': [{'na...",15410125519,2752,15410092449,95779,PushPop,734,734,1,1417,0,33664,33663.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,2275
2276,{'traceMarker': ['/home/llama/example_text_com...,15410496624,2784,15410446827,111189,PushPop,734,734,1,1416,1,33668,6462.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,2276
2277,"{'mod': 'Tensor', 'op': 'copy_', 'args': [{'na...",15410496624,2784,15410463197,94019,PushPop,734,734,1,1417,0,33669,33668.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,2277
2278,{'traceMarker': ['/home/llama/example_text_com...,15455599601,19562456,15410752795,77207331,PushPop,734,734,1,1416,1,33672,6462.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,2278


In [9]:
csv_path = BASE_PATH + "nvtx_gpu_proj_trace_filtered.csv"
df_nvtx_gpu_proj_trace_filtered.to_csv(csv_path, index=False)

In [10]:
# Save top 1000 rows
csv_path = BASE_PATH + "nvtx_gpu_proj_trace_filtered_top1000.csv"
df_nvtx_gpu_proj_trace_filtered.head(1000).to_csv(csv_path, index=False)

In [11]:
csv_path = BASE_PATH + "nvtx_gpu_proj_trace_filtered.csv"
df_nvtx_gpu_proj_trace_filtered = pd.read_csv(csv_path)

# Push Pop Trace

In [12]:
nvtx_pushpop_trace_csv_path = REPORTS_PATH + "nvtx_pushpop_trace.csv"
df_nvtx_pushpop_trace = pd.read_csv(nvtx_pushpop_trace_csv_path)

In [13]:
# Add index column
df_nvtx_pushpop_trace['index'] = np.arange(len(df_nvtx_pushpop_trace)) 
df_nvtx_pushpop_trace

,Start (ns),End (ns),Duration (ns),DurChild (ns),DurNonChild (ns),Name,PID,TID,Lvl,NumChild,RangeId,ParentId,RangeStack,NameTree,index
0,2804473360,71907530415,69103057055,69103045295,11760,{'traceMarker': ['/home/llama/example_text_com...,734,734,0,1,3877,NaN,:3877,{'traceMarker': ['/home/llama/example_text_com...,0
1,2804485120,71907530415,69103045295,69102852026,193269,"{'mod': 'torch', 'op': '__getattr__', 'args': []}",734,734,1,1,3878,3877.0,:3877:3878,"--{'mod': 'torch', 'op': '__getattr__', 'args'...",1
2,2804678389,71907530415,69102852026,69102843486,8540,{'traceMarker': ['/home/llama/example_text_com...,734,734,2,1,3879,3878.0,:3877:3878:3879,----{'traceMarker': ['/home/llama/example_text...,2
3,2804686929,71907530415,69102843486,69102663937,179549,"{'mod': 'torch', 'op': '__getattr__', 'args': []}",734,734,3,1,3880,3879.0,:3877:3878:3879:3880,"------{'mod': 'torch', 'op': '__getattr__', 'a...",3
4,2804866478,71907530415,69102663937,69102652267,11670,{'traceMarker': ['/home/llama/example_text_com...,734,734,4,1,3881,3880.0,:3877:3878:3879:3880:3881,--------{'traceMarker': ['/home/llama/example_...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767251,67363629887,67363630387,500,0,500,"aten::resolve_neg, op_id = 458086",734,734,1416,0,981621,6462.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,----------------------------------------------...,767251
767252,67363632037,67363845936,213899,211919,1980,"aten::to, op_id = 458087",734,734,1416,1,981622,6462.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,----------------------------------------------...,767252
767253,67363633597,67363845516,211919,208019,3900,"aten::_to_copy, op_id = 458088",734,734,1417,2,981623,981622.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,----------------------------------------------...,767253
767254,67363635717,67363640557,4840,0,4840,"aten::empty_strided, op_id = 458089",734,734,1418,0,981624,981623.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,----------------------------------------------...,767254


In [14]:
# Filter df_nvtx_pushpop_trace to only include after start time > 15.9s. It is nanoseconds
# df_nvtx_pushpop_trace_filtered = filter_data_after_time(df_nvtx_pushpop_trace, 'Start (ns)')
df_nvtx_pushpop_trace_filtered = filter_data_between_times(df_nvtx_pushpop_trace, START_TIME, END_TIME, 'Start (ns)')

In [15]:
df_nvtx_pushpop_trace_filtered

,Start (ns),End (ns),Duration (ns),DurChild (ns),DurNonChild (ns),Name,PID,TID,Lvl,NumChild,RangeId,ParentId,RangeStack,NameTree,index
0,2804473360,71907530415,69103057055,69103045295,11760,{'traceMarker': ['/home/llama/example_text_com...,734,734,0,1,3877,NaN,:3877,{'traceMarker': ['/home/llama/example_text_com...,0
1,2804485120,71907530415,69103045295,69102852026,193269,"{'mod': 'torch', 'op': '__getattr__', 'args': []}",734,734,1,1,3878,3877.0,:3877:3878,"--{'mod': 'torch', 'op': '__getattr__', 'args'...",1
2,2804678389,71907530415,69102852026,69102843486,8540,{'traceMarker': ['/home/llama/example_text_com...,734,734,2,1,3879,3878.0,:3877:3878:3879,----{'traceMarker': ['/home/llama/example_text...,2
3,2804686929,71907530415,69102843486,69102663937,179549,"{'mod': 'torch', 'op': '__getattr__', 'args': []}",734,734,3,1,3880,3879.0,:3877:3878:3879:3880,"------{'mod': 'torch', 'op': '__getattr__', 'a...",3
4,2804866478,71907530415,69102663937,69102652267,11670,{'traceMarker': ['/home/llama/example_text_com...,734,734,4,1,3881,3880.0,:3877:3878:3879:3880:3881,--------{'traceMarker': ['/home/llama/example_...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5206,15410446827,15410558016,111189,94019,17170,{'traceMarker': ['/home/llama/example_text_com...,734,734,1416,1,33668,6462.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,----------------------------------------------...,5206
5207,15410463197,15410557216,94019,0,94019,"{'mod': 'Tensor', 'op': 'copy_', 'args': [{'na...",734,734,1417,0,33669,33668.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,----------------------------------------------...,5207
5208,15410752795,15487960126,77207331,77187241,20090,{'traceMarker': ['/home/llama/example_text_com...,734,734,1416,1,33672,6462.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,----------------------------------------------...,5208
5209,15410769615,15487956856,77187241,0,77187241,"{'mod': 'Tensor', 'op': 'copy_', 'args': [{'na...",734,734,1417,0,33673,33672.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,----------------------------------------------...,5209


In [16]:
csv_path = BASE_PATH + "nvtx_pushpop_trace_filtered.csv"
df_nvtx_pushpop_trace_filtered.to_csv(csv_path, index=False)

In [17]:
# Save top 1000 rows
csv_path = BASE_PATH + "nvtx_pushpop_trace_filtered_top1000.csv"
df_nvtx_pushpop_trace_filtered.head(1000).to_csv(csv_path, index=False)

In [18]:
# Load the CSV data into a DataFrame
csv_path = BASE_PATH + "nvtx_pushpop_trace_filtered.csv"
df_nvtx_pushpop_trace_filtered = pd.read_csv(csv_path)

# CUDA API

In [19]:
cuda_api_trace_csv_path = REPORTS_PATH + "cuda_api_trace.csv"
df_cuda_api_trace = pd.read_csv(cuda_api_trace_csv_path)

In [20]:
df_cuda_api_trace

,Start (ns),Duration (ns),Name,Result,CorrID,Pid,Tid,T-Pri,Thread Name
0,3340891939,2470,cuModuleGetLoadingMode,0,1,734,734,20,python
1,3381961391,2258288,cudaGetDeviceProperties_v2_v12000,0,127,734,734,20,python
2,9240404820,15439,cudaStreamIsCapturing_v10000,0,137,734,734,20,python
3,9240428409,556777,cudaMalloc,0,138,734,734,20,python
4,9240428649,556357,cudaMalloc,0,138,734,734,20,python
...,...,...,...,...,...,...,...,...,...
179034,67363811896,3800,cudaStreamSynchronize,0,1075793,734,734,20,python
179035,67363812036,3580,cudaStreamSynchronize,0,1075793,734,734,20,python
179036,67363992815,12210,cudaDeviceSynchronize,0,1075798,734,734,20,python
179037,67363992895,12040,cudaDeviceSynchronize,0,1075798,734,734,20,python


# CUDA GPU Trace

In [21]:
cuda_gpu_trace_csv_path = REPORTS_PATH + "cuda_gpu_trace.csv"
df_cuda_gpu_trace = pd.read_csv(cuda_gpu_trace_csv_path)

In [22]:
# Drop "Device" column
df_cuda_gpu_trace = df_cuda_gpu_trace.drop(columns=["Device"])

df_cuda_gpu_trace.columns

Index(['Start (ns)', 'Duration (ns)', 'CorrId', 'GrdX', 'GrdY', 'GrdZ', 'BlkX',
       'BlkY', 'BlkZ', 'Reg/Trd', 'StcSMem (MB)', 'DymSMem (MB)', 'Bytes (MB)',
       'Throughput (MB/s)', 'SrcMemKd', 'DstMemKd', 'Ctx', 'GreenCtx', 'Strm',
       'Name'],
      dtype='object')

In [23]:
df_cuda_gpu_trace[200:450]

,Start (ns),Duration (ns),CorrId,GrdX,GrdY,GrdZ,BlkX,BlkY,BlkZ,Reg/Trd,StcSMem (MB),DymSMem (MB),Bytes (MB),Throughput (MB/s),SrcMemKd,DstMemKd,Ctx,GreenCtx,Strm,Name
200,10963874588,6850323,5306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.178,13075.743,Pageable,Device,1,NaN,7,[CUDA memcpy Host-to-Device]
201,11001394375,6796659,5315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.178,13256.098,Pageable,Device,1,NaN,7,[CUDA memcpy Host-to-Device]
202,11013875947,2592,5324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.008,3160.490,Pageable,Device,1,NaN,7,[CUDA memcpy Host-to-Device]
203,11014172076,2688,5333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.008,3047.612,Pageable,Device,1,NaN,7,[CUDA memcpy Host-to-Device]
204,11027801523,3324169,5342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.554,10066.330,Pageable,Device,1,NaN,7,[CUDA memcpy Host-to-Device]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,16428832589,6176,7585,128.0,1.0,1.0,128.0,1.0,1.0,16.0,0.000,0.000,NaN,NaN,NaN,NaN,1,NaN,7,"void at::native::elementwise_kernel<(int)128, ..."
446,16429379055,7456,7598,64.0,1.0,1.0,128.0,1.0,1.0,32.0,0.000,0.000,NaN,NaN,NaN,NaN,1,NaN,7,void at::native::unrolled_elementwise_kernel<a...
447,16429933136,7296,7609,64.0,1.0,1.0,128.0,1.0,1.0,16.0,0.000,0.000,NaN,NaN,NaN,NaN,1,NaN,7,"void at::native::elementwise_kernel<(int)128, ..."
448,16684915911,47424,9088,64.0,1.0,1.0,128.0,1.0,1.0,146.0,0.049,0.049,NaN,NaN,NaN,NaN,1,NaN,7,ampere_fp16_s16816gemm_fp16_64x64_sliced1x2_ld...


# CUDA Kernel Trace

In [24]:
cuda_kernel_exec_trace_csv_path = REPORTS_PATH + "cuda_kern_exec_trace.csv"
df_cuda_kernel_exec_trace = pd.read_csv(cuda_kernel_exec_trace_csv_path)

In [25]:
df_cuda_kernel_exec_trace

,API Start (ns),API Dur (ns),Queue Start (ns),Queue Dur (ns),Kernel Start (ns),Kernel Dur (ns),Total Dur (ns),PID,TID,DevId,API Function,GridXYZ,BlockXYZ,Kernel Name
0,9246594615,8518743,9.255113e+09,376.0,9255113734,8896,8528015,734,734,0,cudaLaunchKernel,4096 1 1,128 1 1,void at::native::vectorized_elementwise_kernel...
1,9246594695,8518493,9.255113e+09,546.0,9255113734,8896,8527935,734,734,0,cudaLaunchKernel,4096 1 1,128 1 1,void at::native::vectorized_elementwise_kernel...
2,9255722164,13590,9.255736e+09,2622.0,9255738376,8864,25076,734,734,0,cudaLaunchKernel,4096 1 1,128 1 1,void at::native::vectorized_elementwise_kernel...
3,9255722284,13360,9.255736e+09,2732.0,9255738376,8864,24956,734,734,0,cudaLaunchKernel,4096 1 1,128 1 1,void at::native::vectorized_elementwise_kernel...
4,9260371969,13560,9.260386e+09,2013.0,9260387542,2976,18549,734,734,0,cudaLaunchKernel,8 1 1,128 1 1,void at::native::vectorized_elementwise_kernel...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166043,67362472893,11900,6.736248e+10,1601.0,67362486394,3712,17213,734,734,0,cudaLaunchKernel,1 1 1,128 1 1,void at::native::vectorized_elementwise_kernel...
166044,67362754532,10630,6.736277e+10,1969.0,67362767131,3616,16215,734,734,0,cudaLaunchKernel,1 1 1,128 1 1,void at::native::vectorized_elementwise_kernel...
166045,67362754632,10430,6.736277e+10,2069.0,67362767131,3616,16115,734,734,0,cudaLaunchKernel,1 1 1,128 1 1,void at::native::vectorized_elementwise_kernel...
166046,67363072620,11840,6.736308e+10,1872.0,67363086332,3808,17520,734,734,0,cudaLaunchKernel,1 1 1,128 1 1,void at::native::vectorized_elementwise_kernel...


# Helpers for Summary

In [26]:
def summarize_operations(df):
    """Create a detailed summary for CUDA API operations."""
    # Group by operation type and calculate required statistics till 5 decimal places
    operation_summary = df.groupby('Name').agg(
        Total_Time_ns=('Duration (ns)', lambda x: round(x.sum(), 6)),
        Instances=('Duration (ns)', 'size'),
        Avg_ns=('Duration (ns)', lambda x: round(x.mean(), 6)),
        Med_ns=('Duration (ns)', lambda x: round(x.median(), 6)),
        Min_ns=('Duration (ns)', lambda x: round(x.min(), 6)),
        Max_ns=('Duration (ns)', lambda x: round(x.max(), 6)),
        StdDev_ns=('Duration (ns)', lambda x: round(x.std(), 6))
    ).reset_index()

    # Calculate the total time of all operations to find the percentage of time spent
    total_time = operation_summary['Total_Time_ns'].sum()
    operation_summary['Time_Percent'] = (operation_summary['Total_Time_ns'] / total_time) * 100

    # Reorder and rename columns for the final summary
    operation_summary = operation_summary[[
        'Time_Percent', 'Total_Time_ns', 'Instances', 'Avg_ns', 'Med_ns', 'Min_ns', 'Max_ns', 'StdDev_ns', 'Name'
    ]]
    operation_summary.columns = [
        'Time (%)', 'Total Time (ns)', 'Instances', 'Avg (ns)', 'Med (ns)', 'Min (ns)', 'Max (ns)', 'StdDev (ns)', 'Operation'
    ]

    # Categorize operations
    operation_summary['Category'] = np.where(operation_summary['Operation'].str.contains('cudaMalloc|cudaFree|cudaMemcpy'), 'MEMORY_OPER', 'CUDA_API')

    return operation_summary.sort_values(by='Time (%)', ascending=False)

# Main function to process the data
def process_cuda_api_summary(df_cuda_api_trace):
    # Filter data based on the time after the initial period to ignore
    # filtered_data = filter_data_after_time(df_cuda_api_trace, 'Start (ns)')
    filtered_data = filter_data_between_times(df_cuda_api_trace, START_TIME, END_TIME, 'Start (ns)')

    # Generate the summary
    summary = summarize_operations(filtered_data)

    return summary, filtered_data

In [27]:
# Function to summarize memory operations by size
def summarize_memory_by_size(df, operation_col='Name'):
    # Filter by time
    # df = filter_data_after_time(df, 'Start (ns)')
    df = filter_data_between_times(df, START_TIME, END_TIME, 'Start (ns)')
    # Filter only memory-related operations
    # df = df[df[operation_col].str.contains('memcpy|memset', case=False, na=False)]
    # Filter where Bytes (MB) is not null
    df = df[~df['Bytes (MB)'].isnull()]
    
    # Ensure column names are correct, replace 'Operation' with the actual column name if different
    # memory_summary = df.groupby(operation_col).agg(
    #     Total_MB=('Bytes (MB)', 'sum'),
    #     Count=('Bytes (MB)', 'size'),
    #     Avg_MB=('Bytes (MB)', 'mean'),
    #     Med_MB=('Bytes (MB)', 'median'),
    #     Min_MB=('Bytes (MB)', 'min'),
    #     Max_MB=('Bytes (MB)', 'max'),
    #     StdDev_MB=('Bytes (MB)', 'std')
    # ).reset_index()
    memory_summary = df.groupby(operation_col).agg(
        Total_MB=('Bytes (MB)', lambda x: round(x.sum(), 6)),
        Count=('Bytes (MB)', 'size'),
        Avg_MB=('Bytes (MB)', lambda x: round(x.mean(), 6)),
        Med_MB=('Bytes (MB)', lambda x: round(x.median(), 6)),
        Min_MB=('Bytes (MB)', lambda x: round(x.min(), 6)),
        Max_MB=('Bytes (MB)', lambda x: round(x.max(), 6)),
        StdDev_MB=('Bytes (MB)', lambda x: round(x.std(), 6)
        )
    ).reset_index()
        
    # Get percentage of total memory transferred
    total_memory = memory_summary['Total_MB'].sum()
    memory_summary['Size_Percent'] = (memory_summary['Total_MB'] / total_memory) * 100

    memory_summary.columns = [
        'Operation', 'Total (MB)', 'Count', 'Avg (MB)', 'Med (MB)', 'Min (MB)', 'Max (MB)', 'StdDev (MB)', 'Size_Percent'
    ]

    return memory_summary

In [28]:
# Function to summarize memory operations by time
def summarize_memory_by_time(df, operation_col='Name'):
    # Filter by time
    # df = filter_data_after_time(df, 'Start (ns)')
    df = filter_data_between_times(df, START_TIME, END_TIME, 'Start (ns)')
    # Filter only memory-related operations
    # df = df[df[operation_col].str.contains('memcpy|memset', case=False, na=False)]
    df = df[~df['Bytes (MB)'].isnull()]
    
    # Aggregate and calculate statistics for operation time
    # time_summary = df.groupby(operation_col).agg(
    #     Total_Time_ns=('Duration (ns)', 'sum'),
    #     Count=('Duration (ns)', 'size'),
    #     Avg_ns=('Duration (ns)', 'mean'),
    #     Med_ns=('Duration (ns)', 'median'),
    #     Min_ns=('Duration (ns)', 'min'),
    #     Max_ns=('Duration (ns)', 'max'),
    #     StdDev_ns=('Duration (ns)', 'std')
    # ).reset_index()
    time_summary = df.groupby(operation_col).agg(
        Total_Time_ns=('Duration (ns)', lambda x: round(x.sum(), 6)),
        Count=('Duration (ns)', 'size'),
        Avg_ns=('Duration (ns)', lambda x: round(x.mean(), 6)),
        Med_ns=('Duration (ns)', lambda x: round(x.median(), 6)),
        Min_ns=('Duration (ns)', lambda x: round(x.min(), 6)),
        Max_ns=('Duration (ns)', lambda x: round(x.max(), 6)),
        StdDev_ns=('Duration (ns)', lambda x: round(x.std(), 6)
        )
    ).reset_index()

    # Calculate the total time to find the percentage of time spent
    total_time = time_summary['Total_Time_ns'].sum()
    time_summary['Time_Percent'] = (time_summary['Total_Time_ns'] / total_time) * 100

    time_summary.rename(columns={
        'Time_Percent': 'Time (%)',
        'Total_Time_ns': 'Total Time (ns)',
        'Count': 'Count',
        'Avg_ns': 'Avg (ns)',
        'Med_ns': 'Med (ns)',
        'Min_ns': 'Min (ns)',
        'Max_ns': 'Max (ns)',
        'StdDev_ns': 'StdDev (ns)',
        operation_col: 'Operation'
    }, inplace=True)

    return time_summary

In [29]:
def summarize_kernel_launch_and_exec(df):
    # Filter by time
    # df = filter_data_after_time(df, 'API Start (ns)')
    df = filter_data_between_times(df, START_TIME, END_TIME, 'API Start (ns)')
    
    # Columns
    # API Start (ns)	API Dur (ns)	Queue Start (ns)	Queue Dur (ns)	Kernel Start (ns)	Kernel Dur (ns)	Total Dur (ns)	PID	TID	DevId	API Function	GridXYZ	BlockXYZ	Kernel Name
    
    # Group by PID, TID, DevId, API Function, Kernel Name
    kernel_summary = df.groupby(['PID', 'TID', 'DevId', 'API Function', 'Kernel Name']).agg(
        Count=('API Start (ns)', 'size'),
        QCount=('Queue Dur (ns)', 'size'),
        TAvg=('Total Dur (ns)', 'mean'),
        TMed=('Total Dur (ns)', 'median'),
        TMin=('Total Dur (ns)', 'min'),
        TMax=('Total Dur (ns)', 'max'),
        TStdDev=('Total Dur (ns)', 'std'),
        AAvg=('API Dur (ns)', 'mean'),
        AMed=('API Dur (ns)', 'median'),
        AMin=('API Dur (ns)', 'min'),
        AMax=('API Dur (ns)', 'max'),
        AStdDev=('API Dur (ns)', 'std'),
        QAvg=('Queue Dur (ns)', 'mean'),
        QMed=('Queue Dur (ns)', 'median'),
        QMin=('Queue Dur (ns)', 'min'),
        QMax=('Queue Dur (ns)', 'max'),
        QStdDev=('Queue Dur (ns)', 'std'),
        KAvg=('Kernel Dur (ns)', 'mean'),
        KMed=('Kernel Dur (ns)', 'median'),
        KMin=('Kernel Dur (ns)', 'min'),
        KMax=('Kernel Dur (ns)', 'max'),
        KStdDev=('Kernel Dur (ns)', 'std')
    ).reset_index()
    
    
    # Reorder and rename columns for the final summary
    kernel_summary = kernel_summary[[
        'PID', 'TID', 'DevId', 'Count', 'QCount', 'TAvg', 'TMed', 'TMin', 'TMax', 'TStdDev', 'AAvg', 'AMed', 'AMin', 'AMax', 'AStdDev', 'QAvg', 'QMed', 'QMin', 'QMax', 'QStdDev', 'KAvg', 'KMed', 'KMin', 'KMax', 'KStdDev', 'API Function', 'Kernel Name'
    ]]
    
    kernel_summary.columns = [
        'PID', 'TID', 'DevId', 'Count', 'QCount', 'TAvg (ns)', 'TMed (ns)', 'TMin (ns)', 'TMax (ns)', 'TStdDev (ns)', 'AAvg (ns)', 'AMed (ns)', 'AMin (ns)', 'AMax (ns)', 'AStdDev (ns)', 'QAvg (ns)', 'QMed (ns)', 'QMin (ns)', 'QMax (ns)', 'QStdDev (ns)', 'KAvg (ns)', 'KMed (ns)', 'KMin (ns)', 'KMax (ns)', 'KStdDev (ns)', 'API Function', 'Kernel Name'
    ]
    
    # Sort by Total Average Duration
    kernel_summary = kernel_summary.sort_values(by='TAvg (ns)', ascending=False)
    
    return kernel_summary

# GPU and CPU Times

In [48]:
df_nvtx_gpu_proj_trace_filtered

,Name,Projected Start (ns),Projected Duration (ns),Orig Start (ns),Orig Duration (ns),Style,PID,TID,NumGPUOps,Lvl,NumChild,RangeId,ParentId,RangeStack,index
0,{'traceMarker': ['/home/llama/example_text_com...,9255113734,58108556024,2804473360,69103057055,PushPop,734,734,88119,0,1,3877,NaN,:3877,0
1,"{'mod': 'torch', 'op': '__getattr__', 'args': []}",9255113734,58108556024,2804485120,69103045295,PushPop,734,734,88119,1,1,3878,3877.0,:3877:3878,1
2,{'traceMarker': ['/home/llama/example_text_com...,9255113734,58108556024,2804678389,69102852026,PushPop,734,734,88119,2,1,3879,3878.0,:3877:3878:3879,2
3,"{'mod': 'torch', 'op': '__getattr__', 'args': []}",9255113734,58108556024,2804686929,69102843486,PushPop,734,734,88119,3,1,3880,3879.0,:3877:3878:3879:3880,3
4,{'traceMarker': ['/home/llama/example_text_com...,9255113734,58108556024,2804866478,69102663937,PushPop,734,734,88119,4,1,3881,3880.0,:3877:3878:3879:3880:3881,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,"{'mod': 'Tensor', 'op': 'copy_', 'args': [{'na...",15410125519,2752,15410092449,95779,PushPop,734,734,1,1417,0,33664,33663.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,2275
2276,{'traceMarker': ['/home/llama/example_text_com...,15410496624,2784,15410446827,111189,PushPop,734,734,1,1416,1,33668,6462.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,2276
2277,"{'mod': 'Tensor', 'op': 'copy_', 'args': [{'na...",15410496624,2784,15410463197,94019,PushPop,734,734,1,1417,0,33669,33668.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,2277
2278,{'traceMarker': ['/home/llama/example_text_com...,15455599601,19562456,15410752795,77207331,PushPop,734,734,1,1416,1,33672,6462.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,2278


In [49]:
df_nvtx_gpu_proj_trace_processed = df_nvtx_gpu_proj_trace_filtered.copy()

In [50]:
# Filter data from where Orig Duration (ns) is not greater than END_TIME
df_nvtx_gpu_proj_trace_processed = df_nvtx_gpu_proj_trace_processed[df_nvtx_gpu_proj_trace_processed['Orig Duration (ns)'] <=  END_TIME]

In [51]:
df_nvtx_gpu_proj_trace_processed

,Name,Projected Start (ns),Projected Duration (ns),Orig Start (ns),Orig Duration (ns),Style,PID,TID,NumGPUOps,Lvl,NumChild,RangeId,ParentId,RangeStack,index
1416,{'traceMarker': ['/home/llama/example_text_com...,9255113734,8896,9246149808,9129979,PushPop,734,734,1,1416,1,19305,6462.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,1416
1417,"{'mod': 'torch', 'op': 'zeros', 'args': []}",9255113734,8896,9246160138,9117809,PushPop,734,734,1,1417,0,19306,19305.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,1417
1418,{'traceMarker': ['/home/llama/example_text_com...,9255738376,8864,9255671075,100669,PushPop,734,734,1,1416,1,19329,6462.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,1418
1419,"{'mod': 'torch', 'op': 'zeros', 'args': []}",9255738376,8864,9255682335,88659,PushPop,734,734,1,1417,0,19330,19329.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,1419
1420,{'traceMarker': ['/home/llama/example_text_com...,9260387542,2976,9259587543,838665,PushPop,734,734,1,1416,1,19357,6462.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,1420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2275,"{'mod': 'Tensor', 'op': 'copy_', 'args': [{'na...",15410125519,2752,15410092449,95779,PushPop,734,734,1,1417,0,33664,33663.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,2275
2276,{'traceMarker': ['/home/llama/example_text_com...,15410496624,2784,15410446827,111189,PushPop,734,734,1,1416,1,33668,6462.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,2276
2277,"{'mod': 'Tensor', 'op': 'copy_', 'args': [{'na...",15410496624,2784,15410463197,94019,PushPop,734,734,1,1417,0,33669,33668.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,2277
2278,{'traceMarker': ['/home/llama/example_text_com...,15455599601,19562456,15410752795,77207331,PushPop,734,734,1,1416,1,33672,6462.0,:3877:3878:3879:3880:3881:3882:3883:3884:3887:...,2278


In [52]:
df_nvtx_gpu_proj_trace_processed.to_csv(BASE_PATH + "sample_nvtx_gpu_proj_trace_processed.csv", index=False)

In [53]:
# Separate kernel and memory operations in CUDA GPU trace
df_cuda_gpu_trace['Kernel Ops'] = df_cuda_gpu_trace.apply(lambda row: row['Name'] if pd.isnull(row['Bytes (MB)']) else None, axis=1)
df_cuda_gpu_trace['Memory Ops'] = df_cuda_gpu_trace.apply(lambda row: row['Name'] if not pd.isnull(row['Bytes (MB)']) else None, axis=1)

## Information about CUDA APIs
df_nvtx_gpu_proj_trace_processed['CUDA APIs'] = [[] for _ in range(len(df_nvtx_gpu_proj_trace_processed))]
# Store unique CUDA API Names
df_nvtx_gpu_proj_trace_processed['CUDA API Names'] = [[] for _ in range(len(df_nvtx_gpu_proj_trace_processed))]


## Information about CUDA GPU Kernels
df_nvtx_gpu_proj_trace_processed['Kernels'] = [[] for _ in range(len(df_nvtx_gpu_proj_trace_processed))]
# Store unique Kernel Names
df_nvtx_gpu_proj_trace_processed['Kernel Names'] = [[] for _ in range(len(df_nvtx_gpu_proj_trace_processed))]
# Calculate GPU execution time and idle time for each range
df_nvtx_gpu_proj_trace_processed['GPU Execution Time'] = 0
df_nvtx_gpu_proj_trace_processed['Kernels Match'] = ''
df_nvtx_gpu_proj_trace_processed['Relevant Kernels from GPU Trace'] = [[] for _ in range(len(df_nvtx_gpu_proj_trace_processed))]


## Information about CUDA GPU Memory Operations
df_nvtx_gpu_proj_trace_processed['Memory Ops'] = [[] for _ in range(len(df_nvtx_gpu_proj_trace_processed))]
# Store unique Memory Operation Names
df_nvtx_gpu_proj_trace_processed['Memory Operation Names'] = [[] for _ in range(len(df_nvtx_gpu_proj_trace_processed))]

## Store CorrIDs for Kernels and Memory Operations
df_nvtx_gpu_proj_trace_filtered['GPU Kernel CorrIDs'] = [[] for _ in range(len(df_nvtx_gpu_proj_trace_filtered))]
df_nvtx_gpu_proj_trace_filtered['GPU Memory CorrIDs'] = [[] for _ in range(len(df_nvtx_gpu_proj_trace_filtered))]

## Information about CUDA API to Kernel Mapping
df_nvtx_gpu_proj_trace_processed['CUDA API to Kernel Mapping'] = [{} for _ in range(len(df_nvtx_gpu_proj_trace_processed))]

# Columns for Summary
# Summarize operations for the range
df_nvtx_gpu_proj_trace_processed['CUDA API Summary'] = [{} for _ in range(len(df_nvtx_gpu_proj_trace_processed))]
df_nvtx_gpu_proj_trace_processed['Memory Summary by Size'] = [{} for _ in range(len(df_nvtx_gpu_proj_trace_processed))]
df_nvtx_gpu_proj_trace_processed['Memory Summary by Time'] = [{} for _ in range(len(df_nvtx_gpu_proj_trace_processed))]
df_nvtx_gpu_proj_trace_processed['Kernel Summary'] = [{} for _ in range(len(df_nvtx_gpu_proj_trace_processed))]

In [54]:
# Helper function to calculate idle times
def calculate_idle_times(start_times, end_times):
    idle_times = []
    for i in range(len(start_times) - 1):
        idle_time = start_times[i + 1] - end_times[i]
        if idle_time > 0:
            idle_times.append(idle_time)
    return idle_times

In [55]:
# Helper function to get kernels from df_cuda_gpu_trace
def get_kernels_from_gpu_trace(df, start, end):
    return df[(df['Start (ns)'] >= start) & 
              (df['Start (ns)'] <= end) &
              (df['Kernel Ops'].notnull())]

In [56]:
# Remove all CUDA API calls that are not relevant such as "cuGetProcAddress_v2"
# List of irrelevant CUDA API calls to exclude
irrelevant_cuda_apis = ["cuGetProcAddress_v2"]

In [58]:
# for idx, row in df_nvtx_gpu_proj_trace_filtered.iterrows():
# Loop through the NVTX GPU projection dataframe with a progress bar
for idx, row in tqdm(df_nvtx_gpu_proj_trace_processed.iterrows(), total=len(df_nvtx_gpu_proj_trace_processed), desc="Processing NVTX GPU Projection"):
    range_start = row['Projected Start (ns)']
    range_end = range_start + row['Projected Duration (ns)']
    
    # Filter relevant CUDA API calls
    relevant_cuda_apis = df_cuda_api_trace[(df_cuda_api_trace['Start (ns)'] >= range_start) & 
                                        #    (df_cuda_api_trace['Start (ns)'] + df_cuda_api_trace['Duration (ns)'] <= range_end)]
                                             (df_cuda_api_trace['Start (ns)'] <= range_end)]
                                            #  (df_cuda_api_trace['Start (ns)'] <= range_end) &
                                            #     (~df_cuda_api_trace['Name'].isin(irrelevant_cuda_apis))]
    relevant_cuda_apis = relevant_cuda_apis.drop_duplicates(subset=['CorrID', 'Name', 'Start (ns)'])
    df_nvtx_gpu_proj_trace_processed.at[idx, 'CUDA APIs'] = relevant_cuda_apis.to_dict('records')
    
    # Unique CUDA API names
    unique_cuda_api_names = relevant_cuda_apis['Name'].unique()
    # df_nvtx_gpu_proj_trace_processed.at[idx, 'CUDA API Names'] = unique_cuda_api_names
    # Store as strings with comma separated values
    df_nvtx_gpu_proj_trace_processed.at[idx, 'CUDA API Names'] = ','.join(unique_cuda_api_names)
    
    
    
    # Filter relevant CUDA kernels
    relevant_kernels = df_cuda_kernel_exec_trace[(df_cuda_kernel_exec_trace['Kernel Start (ns)'] >= range_start) & 
                                                #  (df_cuda_kernel_exec_trace['Kernel Start (ns)'] + df_cuda_kernel_exec_trace['Kernel Dur'] <= range_end)]
                                                    (df_cuda_kernel_exec_trace['Kernel Start (ns)'] <= range_end)]
    relevant_kernels = relevant_kernels.drop_duplicates(subset=['Kernel Start (ns)', 'Kernel Name'])
    df_nvtx_gpu_proj_trace_processed.at[idx, 'Kernels'] = relevant_kernels.to_dict('records')
    
    # Unique Kernel names
    unique_kernel_names = relevant_kernels['Kernel Name'].unique()
    # df_nvtx_gpu_proj_trace_processed.at[idx, 'Kernel Names'] = unique_kernel_names
    # Store as strings with comma separated values
    df_nvtx_gpu_proj_trace_processed.at[idx, 'Kernel Names'] = ','.join(unique_kernel_names)
    
    # Sum of kernel durations
    gpu_execution_time = relevant_kernels['Kernel Dur (ns)'].sum()
    df_nvtx_gpu_proj_trace_processed.at[idx, 'GPU Execution Time'] = gpu_execution_time
    
    # # Calculate GPU idle time within the range
    # kernel_start_times = relevant_kernels['Kernel Start (ns)'].sort_values().values
    # kernel_end_times = (relevant_kernels['Kernel Start (ns)'] + relevant_kernels['Kernel Dur (ns)']).sort_values().values
    # gpu_idle_times = calculate_idle_times(kernel_start_times, kernel_end_times)
    # df_nvtx_gpu_proj_trace_filtered.at[idx, 'GPU Idle Time'] = sum(gpu_idle_times)
    
    # Get kernels from df_cuda_gpu_trace
    relevant_kernels_from_gpu_trace = get_kernels_from_gpu_trace(df_cuda_gpu_trace, range_start, range_end)
    relevant_kernels_from_gpu_trace = relevant_kernels_from_gpu_trace.drop_duplicates(subset=['Start (ns)', 'Name'])
    # df_nvtx_gpu_proj_trace_filtered.at[idx, 'Relevant Kernels from GPU Trace'] = relevant_kernels_from_gpu_trace.to_dict('records')
    
    # Check if the kernels match
    kernels_match = set(relevant_kernels['Kernel Start (ns)']) == set(relevant_kernels_from_gpu_trace['Start (ns)'])
    df_nvtx_gpu_proj_trace_processed.at[idx, 'Kernels Match'] = 'Yes' if kernels_match else 'No'
    
    # Store relevant kernels from GPU trace only if they don't match
    if not kernels_match:
        df_nvtx_gpu_proj_trace_processed.at[idx, 'Relevant Kernels from GPU Trace'] = relevant_kernels_from_gpu_trace.to_dict('records')
    
    # Filter relevant memory operations
    relevant_memory_ops = df_cuda_gpu_trace[(df_cuda_gpu_trace['Start (ns)'] >= range_start) & 
                                            (df_cuda_gpu_trace['Start (ns)'] + df_cuda_gpu_trace['Duration (ns)'] <= range_end) & 
                                            (~df_cuda_gpu_trace['Bytes (MB)'].isnull())]
    relevant_memory_ops = relevant_memory_ops.drop_duplicates(subset=['Start (ns)', 'Name'])
    df_nvtx_gpu_proj_trace_processed.at[idx, 'Memory Ops'] = relevant_memory_ops.to_dict('records')
    
    # Unique Memory Operation names
    unique_memory_op_names = relevant_memory_ops['Name'].unique()
    # df_nvtx_gpu_proj_trace_processed.at[idx, 'Memory Operation Names'] = unique_memory_op_names
    # Store as strings with comma separated values
    df_nvtx_gpu_proj_trace_processed.at[idx, 'Memory Operation Names'] = ','.join(unique_memory_op_names)
    
    # Create a mapping between CUDA API calls and kernels based on CorrId within the range
    api_kernel_mapping = []
    # for _, api_row in relevant_cuda_apis.iterrows():
    #     api_corr_id = api_row['CorrID']
    #     mapped_kernels = df_cuda_gpu_trace[df_cuda_gpu_trace['CorrId'] == api_corr_id].to_dict('records')
    #     api_kernel_mapping.append({
    #         'CUDA API': api_row.to_dict(),
    #         'Kernels': mapped_kernels
    #     })
    for _, api_row in relevant_cuda_apis.iterrows():
        api_corr_id = api_row['CorrID']
        api_name = api_row['Name']
        mapped_kernels = df_cuda_gpu_trace[df_cuda_gpu_trace['CorrId'] == api_corr_id]['Name'].tolist()
        api_kernel_mapping.append({
            'CorrID': api_corr_id,
            'CUDA API': api_name,
            'Kernels': mapped_kernels
        })
    df_nvtx_gpu_proj_trace_processed.at[idx, 'CUDA API to Kernel Mapping'] = api_kernel_mapping

    # # Unique CorrIDs
    # unique_corr_ids = relevant_cuda_apis['CorrID'].unique()
    # df_nvtx_gpu_proj_trace_filtered.at[idx, 'CorrIDs'] = unique_corr_ids
    
    # # GPU CorrIDs
    if relevant_kernels_from_gpu_trace.empty:
        gpu_kernel_corr_ids = []
    else:
        gpu_kernel_corr_ids = relevant_kernels_from_gpu_trace['CorrId'].unique()
        # df_nvtx_gpu_proj_trace_processed.at[idx, 'GPU Kernel CorrIDs'] = gpu_kernel_corr_ids
        # Store as strings with comma separated values
        df_nvtx_gpu_proj_trace_processed.at[idx, 'GPU Kernel CorrIDs'] = ','.join(map(str, gpu_kernel_corr_ids))
    
    # GPU Memory CorrIDs
    if relevant_memory_ops.empty:
        gpu_memory_corr_ids = []
    else:
        gpu_memory_corr_ids = relevant_memory_ops['CorrId'].unique()
        # df_nvtx_gpu_proj_trace_processed.at[idx, 'GPU Memory CorrIDs'] = gpu_memory_corr_ids
        # Store as strings with comma separated values
        df_nvtx_gpu_proj_trace_processed.at[idx, 'GPU Memory CorrIDs'] = ','.join(map(str, gpu_memory_corr_ids))
    
    # Summarize operations for the range
    df_nvtx_gpu_proj_trace_processed.at[idx, 'CUDA API Summary'] = summarize_operations(relevant_cuda_apis).to_dict('records')
    df_nvtx_gpu_proj_trace_processed.at[idx, 'Memory Summary by Size'] = summarize_memory_by_size(relevant_memory_ops).to_dict('records')
    df_nvtx_gpu_proj_trace_processed.at[idx, 'Memory Summary by Time'] = summarize_memory_by_time(relevant_memory_ops).to_dict('records')
    df_nvtx_gpu_proj_trace_processed.at[idx, 'Kernel Summary'] = summarize_kernel_launch_and_exec(relevant_kernels).to_dict('records')
    
    # Save if idx is 100 or 1000
    if idx == 100 or idx == 1000:
        csv_path = BASE_PATH + f"test_nvtx_gpu_proj_trace_processed_{idx}.csv"
        # df_nvtx_gpu_proj_trace_processed.to_csv(csv_path, index=False)
        df_nvtx_gpu_proj_trace_processed.head(idx).to_csv(csv_path, index=False)
        
    # if idx > 10:
    #     break

Processing NVTX GPU Projection: 100%|██████████| 864/864 [00:13<00:00, 61.98it/s]


### Using multiprocessing

In [77]:
from joblib import Parallel, delayed
import multiprocessing as mp

### Save results

In [59]:
df_nvtx_gpu_proj_trace_processed.head(200)

,Name,Projected Start (ns),Projected Duration (ns),Orig Start (ns),Orig Duration (ns),Style,PID,TID,NumGPUOps,Lvl,...,Relevant Kernels from GPU Trace,Memory Ops,Memory Operation Names,CUDA API to Kernel Mapping,CUDA API Summary,Memory Summary by Size,Memory Summary by Time,Kernel Summary,GPU Kernel CorrIDs,GPU Memory CorrIDs
1416,{'traceMarker': ['/home/llama/example_text_com...,9255113734,8896,9246149808,9129979,PushPop,734,734,1,1416,...,[],[],,[],[],[],[],"[{'PID': 734, 'TID': 734, 'DevId': 0, 'Count':...",201,nan
1417,"{'mod': 'torch', 'op': 'zeros', 'args': []}",9255113734,8896,9246160138,9117809,PushPop,734,734,1,1417,...,[],[],,[],[],[],[],"[{'PID': 734, 'TID': 734, 'DevId': 0, 'Count':...",201,nan
1418,{'traceMarker': ['/home/llama/example_text_com...,9255738376,8864,9255671075,100669,PushPop,734,734,1,1416,...,[],[],,[],[],[],[],"[{'PID': 734, 'TID': 734, 'DevId': 0, 'Count':...",218,nan
1419,"{'mod': 'torch', 'op': 'zeros', 'args': []}",9255738376,8864,9255682335,88659,PushPop,734,734,1,1417,...,[],[],,[],[],[],[],"[{'PID': 734, 'TID': 734, 'DevId': 0, 'Count':...",218,nan
1420,{'traceMarker': ['/home/llama/example_text_com...,9260387542,2976,9259587543,838665,PushPop,734,734,1,1416,...,[],[],,[],[],[],[],"[{'PID': 734, 'TID': 734, 'DevId': 0, 'Count':...",271,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1611,"{'mod': 'torch', 'op': 'zeros', 'args': []}",9416804821,8544,9416772350,64999,PushPop,734,734,1,1417,...,[],[],,[],[],[],[],"[{'PID': 734, 'TID': 734, 'DevId': 0, 'Count':...",3546,nan
1612,{'traceMarker': ['/home/llama/example_text_com...,9419462173,3232,9419414825,77109,PushPop,734,734,1,1416,...,[],[],,[],[],[],[],"[{'PID': 734, 'TID': 734, 'DevId': 0, 'Count':...",3595,nan
1613,"{'mod': 'torch', 'op': 'ones', 'args': []}",9419462173,3232,9419422385,68909,PushPop,734,734,1,1417,...,[],[],,[],[],[],[],"[{'PID': 734, 'TID': 734, 'DevId': 0, 'Count':...",3595,nan
1614,{'traceMarker': ['/home/llama/example_text_com...,9419680093,2944,9419641634,73849,PushPop,734,734,1,1416,...,[],[],,[],[],[],[],"[{'PID': 734, 'TID': 734, 'DevId': 0, 'Count':...",3608,nan


In [60]:
df_nvtx_gpu_proj_trace_processed.head(200).to_csv(BASE_PATH + "nvtx_gpu_proj_trace_processed_sample.csv", index=False)

In [61]:
# Save processed data
csv_path = BASE_PATH + "nvtx_gpu_proj_trace_processed.csv"
df_nvtx_gpu_proj_trace_processed.to_csv(csv_path, index=False)

# Summaries

## CUDA Summaries

#### CUDA API Summary

In [62]:
# Generate the summary
df_cuda_api_summary, df_cuda_api_trace_filtered = process_cuda_api_summary(df_cuda_api_trace)

In [63]:
df_cuda_api_summary

,Time (%),Total Time (ns),Instances,Avg (ns),Med (ns),Min (ns),Max (ns),StdDev (ns),Operation,Category
5,86.831550,2375507047,582,4.081627e+06,3171792.0,9510,19638641,3.206453e+06,cudaMemcpyAsync,MEMORY_OPER
3,8.148573,222925793,280,7.961635e+05,13420.0,9860,44947910,4.365736e+06,cudaLaunchKernel,CUDA_API
4,4.637789,126879000,480,2.643312e+05,239078.5,209729,1102723,9.686421e+04,cudaMalloc,MEMORY_OPER
8,0.284093,7772116,582,1.335415e+04,14110.0,3950,259159,1.499185e+04,cudaStreamSynchronize,CUDA_API
2,0.082547,2258288,1,2.258288e+06,2258288.0,2258288,2258288,NaN,cudaGetDeviceProperties_v2_v12000,CUDA_API
7,0.013657,373637,240,1.556821e+03,1450.0,1300,15439,1.028034e+03,cudaStreamIsCapturing_v10000,CUDA_API
1,0.000950,25990,2,1.299500e+04,12995.0,12900,13090,1.343503e+02,cudaDeviceSynchronize,CUDA_API
6,0.000751,20550,1,2.055000e+04,20550.0,20550,20550,NaN,cudaProfilerStart,CUDA_API
0,0.000090,2470,1,2.470000e+03,2470.0,2470,2470,NaN,cuModuleGetLoadingMode,CUDA_API


In [64]:
df_cuda_api_trace_filtered

,Start (ns),Duration (ns),Name,Result,CorrID,Pid,Tid,T-Pri,Thread Name
0,3340891939,2470,cuModuleGetLoadingMode,0,1,734,734,20,python
1,3381961391,2258288,cudaGetDeviceProperties_v2_v12000,0,127,734,734,20,python
2,9240404820,15439,cudaStreamIsCapturing_v10000,0,137,734,734,20,python
3,9240428409,556777,cudaMalloc,0,138,734,734,20,python
4,9240428649,556357,cudaMalloc,0,138,734,734,20,python
...,...,...,...,...,...,...,...,...,...
2164,15475223807,16780,cudaStreamSynchronize,0,7377,734,734,20,python
2165,15475225257,15160,cudaStreamSynchronize,0,7377,734,734,20,python
2166,15491130778,20550,cudaProfilerStart,0,7382,734,734,20,python
2167,15491371587,13090,cudaDeviceSynchronize,0,7385,734,734,20,python


In [65]:
# Save to "generated_reports" folder
cupa_api_summary_csv_path = BASE_PATH + "cuda_api_summary.csv"
df_cuda_api_summary.to_csv(cupa_api_summary_csv_path, index=False)

In [66]:
df_cuda_api_summary

,Time (%),Total Time (ns),Instances,Avg (ns),Med (ns),Min (ns),Max (ns),StdDev (ns),Operation,Category
5,86.831550,2375507047,582,4.081627e+06,3171792.0,9510,19638641,3.206453e+06,cudaMemcpyAsync,MEMORY_OPER
3,8.148573,222925793,280,7.961635e+05,13420.0,9860,44947910,4.365736e+06,cudaLaunchKernel,CUDA_API
4,4.637789,126879000,480,2.643312e+05,239078.5,209729,1102723,9.686421e+04,cudaMalloc,MEMORY_OPER
8,0.284093,7772116,582,1.335415e+04,14110.0,3950,259159,1.499185e+04,cudaStreamSynchronize,CUDA_API
2,0.082547,2258288,1,2.258288e+06,2258288.0,2258288,2258288,NaN,cudaGetDeviceProperties_v2_v12000,CUDA_API
7,0.013657,373637,240,1.556821e+03,1450.0,1300,15439,1.028034e+03,cudaStreamIsCapturing_v10000,CUDA_API
1,0.000950,25990,2,1.299500e+04,12995.0,12900,13090,1.343503e+02,cudaDeviceSynchronize,CUDA_API
6,0.000751,20550,1,2.055000e+04,20550.0,20550,20550,NaN,cudaProfilerStart,CUDA_API
0,0.000090,2470,1,2.470000e+03,2470.0,2470,2470,NaN,cuModuleGetLoadingMode,CUDA_API


In [67]:
cuda_api_trace_filtered_csv_path = BASE_PATH + "cuda_api_trace_filtered.csv"
df_cuda_api_trace_filtered.to_csv(cuda_api_trace_filtered_csv_path, index=False)

#### Memops Summaries

In [68]:
df_cuda_memtime_summary = summarize_memory_by_time(df_cuda_gpu_trace)
df_cuda_memsize_summary = summarize_memory_by_size(df_cuda_gpu_trace)

In [69]:
df_cuda_memsize_summary

,Operation,Total (MB),Count,Avg (MB),Med (MB),Min (MB),Max (MB),StdDev (MB),Size_Percent
0,[CUDA memcpy Host-to-Device],13476.808,291,46.312055,33.554,0.008,262.144,38.867176,100.0


In [70]:
df_cuda_memtime_summary

,Operation,Total Time (ns),Count,Avg (ns),Med (ns),Min (ns),Max (ns),StdDev (ns),Time (%)
0,[CUDA memcpy Host-to-Device],1166380807,291,4.008181e+06,3075528.0,2560,19562456,3.189449e+06,100.0


In [71]:
# Save to "generated_reports" folder
cuda_memtime_summary_csv_path = BASE_PATH + "cuda_memtime_summary.csv"
df_cuda_memtime_summary.to_csv(cuda_memtime_summary_csv_path, index=False)

cuda_memsize_summary_csv_path = BASE_PATH + "cuda_memsize_summary.csv"
df_cuda_memsize_summary.to_csv(cuda_memsize_summary_csv_path, index=False)

## Kernel Launch and Exec Time Summaries

In [72]:
df_cuda_kernel_launch_and_exec_time = summarize_kernel_launch_and_exec(df_cuda_kernel_exec_trace)

In [73]:
df_cuda_kernel_launch_and_exec_time

,PID,TID,DevId,Count,QCount,TAvg (ns),TMed (ns),TMin (ns),TMax (ns),TStdDev (ns),...,QMin (ns),QMax (ns),QStdDev (ns),KAvg (ns),KMed (ns),KMin (ns),KMax (ns),KStdDev (ns),API Function,Kernel Name
1,734,734,0,2,2,4.495193e+07,44951927.0,44951882,44951972,6.363961e+01,...,NaN,NaN,NaN,6464.000000,6464.0,6464,6464,0.000000,cudaLaunchKernel,"void at::native::elementwise_kernel<(int)128, ..."
3,734,734,0,2,2,1.403741e+07,14037411.0,14037356,14037466,7.778175e+01,...,NaN,NaN,NaN,4736.000000,4736.0,4736,4736,0.000000,cudaLaunchKernel,void at::native::unrolled_elementwise_kernel<a...
10,734,734,0,2,2,1.369692e+07,13696924.0,13696824,13697024,1.414214e+02,...,NaN,NaN,NaN,3776.000000,3776.0,3776,3776,0.000000,cudaLaunchKernel,void at::native::vectorized_elementwise_kernel...
6,734,734,0,2,2,1.171042e+07,11710422.0,11710347,11710497,1.060660e+02,...,NaN,NaN,NaN,3392.000000,3392.0,3392,3392,0.000000,cudaLaunchKernel,void at::native::vectorized_elementwise_kernel...
5,734,734,0,2,2,1.073470e+07,10734697.0,10734647,10734747,7.071068e+01,...,NaN,NaN,NaN,3392.000000,3392.0,3392,3392,0.000000,cudaLaunchKernel,void at::native::vectorized_elementwise_kernel...
4,734,734,0,2,2,2.866846e+06,2866846.0,2866781,2866911,9.192388e+01,...,1868.0,2088.0,155.563492,5248.000000,5248.0,5248,5248,0.000000,cudaLaunchKernel,void at::native::vectorized_elementwise_kernel...
0,734,734,0,4,4,1.573768e+06,1573770.5,19119,3128412,1.795108e+06,...,1451.0,2209.0,381.863414,3424.000000,3424.0,3328,3520,110.851252,cudaLaunchKernel,void <unnamed>::elementwise_kernel_with_index<...
2,734,734,0,2,2,1.681470e+05,168147.0,168107,168187,5.656854e+01,...,2314.0,2524.0,148.492424,9504.000000,9504.0,9504,9504,0.000000,cudaLaunchKernel,"void at::native::elementwise_kernel<(int)128, ..."
7,734,734,0,258,258,8.660874e+04,21377.0,15085,8528015,7.475775e+05,...,376.0,4561.0,864.982712,5808.364341,3232.0,2944,9024,2786.781497,cudaLaunchKernel,void at::native::vectorized_elementwise_kernel...
8,734,734,0,2,2,4.717600e+04,47176.0,47111,47241,9.192388e+01,...,2033.0,2143.0,77.781746,3168.000000,3168.0,3168,3168,0.000000,cudaLaunchKernel,void at::native::vectorized_elementwise_kernel...


In [74]:
# Save to "generated_reports" folder
cuda_kernel_summary_csv_path = BASE_PATH + "cuda_kernel_launch_and_exec_time_summary.csv"
df_cuda_kernel_launch_and_exec_time.to_csv(cuda_kernel_summary_csv_path, index=False)

### Kernel Summary

In [79]:
def summarize_kernel(df):
    # Filter by time
    # df = filter_data_after_time(df, 'API Start (ns)')
    df = filter_data_between_times(df, START_TIME, END_TIME, 'API Start (ns)')
    
    # Columns
    # API Start (ns)	API Dur (ns)	Queue Start (ns)	Queue Dur (ns)	Kernel Start (ns)	Kernel Dur (ns)	Total Dur (ns)	PID	TID	DevId	API Function	GridXYZ	BlockXYZ	Kernel Name
    
    # Get the summary statistics for the kernel duration
    kernel_summary = df.groupby('Kernel Name').agg(
        Total_Time_ns=('Kernel Dur (ns)', 'sum'),
        Instances=('Kernel Dur (ns)', 'size'),
        Avg_ns=('Kernel Dur (ns)', 'mean'),
        Med_ns=('Kernel Dur (ns)', 'median'),
        Min_ns=('Kernel Dur (ns)', 'min'),
        Max_ns=('Kernel Dur (ns)', 'max'),
        StdDev_ns=('Kernel Dur (ns)', 'std')
    ).reset_index()
    
    # Time (%)	Total Time (ns)	Instances	Avg (ns)	Med (ns)	Min (ns)	Max (ns)	StdDev (ns)	Name
    # Calculate the total time of all operations to find the percentage of time spent
    total_time = kernel_summary['Total_Time_ns'].sum()
    
    kernel_summary['Time (%)'] = (kernel_summary['Total_Time_ns'] / total_time) * 100
    
    # Reorder and rename columns for the final summary
    kernel_summary = kernel_summary[[
        'Time (%)', 'Total_Time_ns', 'Instances', 'Avg_ns', 'Med_ns', 'Min_ns', 'Max_ns', 'StdDev_ns', 'Kernel Name'
    ]]
    
    return kernel_summary.sort_values(by='Total_Time_ns', ascending=False)
    

In [80]:
df_cuda_kernel_summary = summarize_kernel(df_cuda_kernel_exec_trace)

In [ ]:
df_cuda_kernel_summary.head(10)

,Time (%),Total_Time_ns,Instances,Avg_ns,Med_ns,Min_ns,Max_ns,StdDev_ns,Kernel Name
3,38.601166,902228670,20480,44054.134277,35040.0,34176,83776,18129.193032,ampere_fp16_s16816gemm_fp16_64x64_sliced1x2_ld...
1,26.256146,613687360,8064,76102.103175,76000.0,73120,83424,1036.731323,ampere_fp16_s16816gemm_fp16_64x64_ldg8_f2f_sta...
35,6.434864,150402690,20736,7253.216146,6848.0,5440,9312,814.015345,void at::native::unrolled_elementwise_kernel<a...
33,6.077458,142048996,20608,6892.905474,6816.0,5376,8288,632.052543,void at::native::unrolled_elementwise_kernel<a...
25,3.167862,74042746,8320,8899.368510,8800.0,8096,16928,899.338032,"void at::native::reduce_kernel<(int)512, (int)..."
47,2.074047,48476886,12286,3945.701286,3935.0,3648,4864,196.016493,void at::native::vectorized_elementwise_kernel...
20,1.928237,45068864,8320,5416.930769,5408.0,5023,6336,104.975411,"void at::native::elementwise_kernel<(int)128, ..."
18,1.838804,42978534,8192,5246.403076,5280.0,4992,5792,193.659124,"void at::native::elementwise_kernel<(int)128, ..."
50,1.432888,33491014,8192,4088.258545,4096.0,3680,4481,94.495099,void at::native::vectorized_elementwise_kernel...
55,1.367275,31957452,8320,3841.039904,3904.0,3296,4160,133.638455,void at::native::vectorized_elementwise_kernel...


In [81]:
# Save to "generated_reports" folder
cuda_kernel_summary_csv_path = BASE_PATH + "cuda_kernel_summary.csv"
df_cuda_kernel_summary.to_csv(cuda_kernel_summary_csv_path, index=False)

#### NVXT GPU Projection Summary

In [84]:
def generate_nvtx_gpu_projection_summary(df_nvtx):
    # Assuming df_nvtx has columns such as: Range, Style, Total Proj Time (ns), etc.
    # Filter data based on the time after the initial period to ignore
    # df_nvtx = filter_data_after_time(df_nvtx, 'Orig Start (ns)')
    df_nvtx = filter_data_between_times(df_nvtx, START_TIME, END_TIME, 'Orig Start (ns)')
    # Group by 'Range' and 'Style' and calculate statistics
    projection_summary = df_nvtx.groupby(['Name', 'Style']).agg(
        Total_Proj_Time_ns=('Projected Duration (ns)', 'sum'),
        Total_Range_Time_ns=('Orig Duration (ns)', 'sum'),
        Range_Instances=('RangeId', 'nunique'),
        Proj_Avg_ns=('Projected Duration (ns)', 'mean'),
        Proj_Med_ns=('Projected Duration (ns)', 'median'),
        Proj_Min_ns=('Projected Duration (ns)', 'min'),
        Proj_Max_ns=('Projected Duration (ns)', 'max'),
        Proj_StdDev_ns=('Projected Duration (ns)', 'std'),
        Total_GPU_Ops=('NumGPUOps', 'sum'),
        Avg_GPU_Ops=('NumGPUOps', 'mean'),
        Avg_Range_Lvl=('Lvl', 'mean'),
        Avg_Num_Child=('NumChild', 'mean')
    ).reset_index()

    # Handling NaN values for standard deviation in cases of single data point
    projection_summary['Proj_StdDev_ns'].fillna(0, inplace=True)

    # Additional formatting or calculations can be added here as needed

    return projection_summary

In [85]:
df_nvtx_gpu_proj_summary = generate_nvtx_gpu_projection_summary(df_nvtx_gpu_proj_trace_filtered)

In [86]:
# Save to "generated_reports" folder
nvtx_gpu_proj_summary_csv_path = BASE_PATH + "nvtx_gpu_proj_summary.csv"
df_nvtx_gpu_proj_summary.to_csv(nvtx_gpu_proj_summary_csv_path, index=False)

#### NVTX Push Pop

In [87]:
def summarize_nvtx_pushpop(df):
    # Filter data based on the time after the initial period to ignore
    # df = filter_data_after_time(df, 'Start (ns)')
    df = filter_data_between_times(df, START_TIME, END_TIME, 'Start (ns)')
    # Calculate total duration and create a derived column for it
    df['Duration (ns)'] = df['End (ns)'] - df['Start (ns)']

    # Group by 'Range' and calculate the necessary statistics
    summary = df.groupby('Name').agg(
        Total_Time_ns=('Duration (ns)', lambda x: round(x.sum(), 6)),
        Instances=('Duration (ns)', 'size'),
        Avg_ns=('Duration (ns)', lambda x: round(x.mean(), 6)),
        Med_ns=('Duration (ns)', lambda x: round(x.median(), 6)),
        Min_ns=('Duration (ns)', lambda x: round(x.min(), 6)),
        Max_ns=('Duration (ns)', lambda x: round(x.max(), 6)),
        StdDev_ns=('Duration (ns)', lambda x: round(x.std(), 6))
    ).reset_index()

    # Calculate total time across all operations to find the percentage of time spent
    total_time = summary['Total_Time_ns'].sum()
    summary['Time_Percent'] = (summary['Total_Time_ns'] / total_time) * 100

    # Handling NaN values for standard deviation in cases of single data point
    summary['StdDev_ns'].fillna(0, inplace=True)

    # Format the output to match your desired output
    summary = summary[['Time_Percent', 'Total_Time_ns', 'Instances', 'Avg_ns', 'Med_ns', 'Min_ns', 'Max_ns', 'StdDev_ns', 'Name']]
    summary.columns = ['Time (%)', 'Total Time (ns)', 'Instances', 'Avg (ns)', 'Med (ns)', 'Min (ns)', 'Max (ns)', 'StdDev (ns)', 'Range']
    
    return summary

In [88]:
df_nvtx_pushpop_summary = summarize_nvtx_pushpop(df_nvtx_pushpop_trace_filtered)

In [89]:
# Save to "generated_reports" folder
nvtx_pushpop_summary_csv_path = BASE_PATH + "nvtx_pushpop_summary.csv"
df_nvtx_pushpop_summary.to_csv(nvtx_pushpop_summary_csv_path, index=False)